# Analyze Faithfulness Vector Per-Token Projections

This notebook loads a learned CoT faithfulness vector and visualizes per-token projections for one faithful pair and one unfaithful pair of CoTs (complementary questions).

In [2]:
# Optional: install plotting deps if missing
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
except Exception:
    import sys
    !{sys.executable} -m pip install matplotlib seaborn -q
    import matplotlib.pyplot as plt
    import seaborn as sns
sns.set_context('talk')
sns.set_style('whitegrid')
print('Plot libs ready')

Plot libs ready


In [3]:
import os, glob, time, math, torch
from typing import Optional, Tuple
from faithvec.model import HFModel
from faithvec.wm import load_wm_pairs_list
from faithvec.utils import parse_yes_no
print('Imports OK')

/opt/anaconda3/envs/unfaithfulcots/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imports OK


In [4]:
# Config: pick a learned vector file (defaults to most recent under data/faithvec)
vec_dir = 'data/faithvec'
vec_path = None  # set to a specific .pt if you want

def latest_vector(path=vec_dir):
    files = sorted(glob.glob(os.path.join(path, '*.pt')), key=os.path.getmtime)
    return files[-1] if files else None

if vec_path is None:
    vec_path = latest_vector()
vec_path


'data/faithvec/deepseek-ai-DeepSeek-R1-Distill-Qwen-1-5B__wm-us-city-lat__prop-latitude__n-20__seed-0__20250824-231048.pt'

In [5]:
assert vec_path and os.path.exists(vec_path), f'No vector file found at {vec_path}'
payload = torch.load(vec_path, map_location='cpu')
v = payload['v'].float()
mu_f = payload.get('mu_f')
mu_u = payload.get('mu_u')
meta = payload.get('meta', {})
v = v / (v.norm(p=2) + 1e-8)
print('Loaded vector:', vec_path)
print('Model:', meta.get('model'))
print('WM path:', meta.get('wm_path'))
print('Property:', meta.get('wm_prop'))
print('Train n:', meta.get('n_train'), 'Test n:', meta.get('n_test'))

Loaded vector: data/faithvec/deepseek-ai-DeepSeek-R1-Distill-Qwen-1-5B__wm-us-city-lat__prop-latitude__n-20__seed-0__20250824-231048.pt
Model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
WM path: data/wm_cache/wm-us-city-lat.yaml
Property: latitude
Train n: 20 Test n: 10


/var/folders/s1/tvdnvl8n0l52wc16d3x78qlw0000gn/T/ipykernel_98614/1139944064.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  payload = torch.load(vec_path, map_location='

In [6]:
# Initialize the same model for consistent hidden sizes
model_name = meta.get('model', 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
hf = HFModel(model_name=model_name, device=device, device_map=None)
print('HF model ready on', hf.device)
v_dev = v.to(hf.model.device)
v_dev = v_dev / (v_dev.norm(p=2) + 1e-8)

HF model ready on cpu


In [7]:
def generate_with_token_projections(prompt: str, max_new_tokens=64, temperature=0.7):
    tok = hf.tokenizer(prompt, return_tensors='pt').to(hf.model.device)
    gen = hf.model.generate(**tok, max_new_tokens=max_new_tokens, temperature=temperature, do_sample=True, pad_token_id=hf.tokenizer.eos_token_id, return_dict_in_generate=True)
    ids = gen.sequences
    text = hf.tokenizer.decode(ids[0], skip_special_tokens=True)
    out = hf.model(ids, output_hidden_states=True, use_cache=False)
    last = out.hidden_states[-1]  # (1, T, H)
    gen_len = ids.size(1) - tok.input_ids.size(1)
    if gen_len <= 0:
        return text, [], ids
    gen_h = last[:, -gen_len:, :]  # (1, G, H)
    # per-token projection onto v
    projs = (gen_h @ v_dev.view(-1, 1)).squeeze(-1).squeeze(0).detach().float().cpu().tolist()
    return text, projs, ids

def pair_status(ans1: Optional[bool], ans2: Optional[bool]) -> Optional[bool]:
    if ans1 is None or ans2 is None:
        return None
    return (ans1 and not ans2) or (ans2 and not ans1)

In [8]:
# Search WM pairs until we find one faithful and one unfaithful pair
wm_path = meta.get('wm_path', 'data/wm_cache/wm-us-city-lat.yaml')
wm_prop = meta.get('wm_prop', 'latitude')
pairs = load_wm_pairs_list(wm_path, seed=0, property_name=wm_prop)
faithful_ex = None
unfaithful_ex = None
max_tries = 200
for i, p in enumerate(pairs[:max_tries], 1):
    t1, proj1, ids1 = generate_with_token_projections(p.q1, max_new_tokens=64, temperature=0.5)
    t2, proj2, ids2 = generate_with_token_projections(p.q2, max_new_tokens=64, temperature=0.5)
    a1 = parse_yes_no(t1)
    a2 = parse_yes_no(t2)
    st = pair_status(a1, a2)
    if st is True and faithful_ex is None:
        faithful_ex = (p, (t1, proj1), (t2, proj2), (a1, a2))
    if st is False and unfaithful_ex is None:
        unfaithful_ex = (p, (t1, proj1), (t2, proj2), (a1, a2))
    if faithful_ex and unfaithful_ex:
        break
print('Found faithful:', faithful_ex is not None, '| unfaithful:', unfaithful_ex is not None)

faithful_ex, unfaithful_ex

KeyboardInterrupt: 

In [ ]:
# Combined grid plot for faithful and unfaithful pairs
def plot_grid(faithful_ex, unfaithful_ex):
    cases = []
    if faithful_ex is not None:
        cases.append(('Faithful', faithful_ex))
    if unfaithful_ex is not None:
        cases.append(('Unfaithful', unfaithful_ex))
    if not cases:
        print('No cases to plot')
        return
    rows = len(cases)
    cols = 2
    fig, axes = plt.subplots(rows, cols, figsize=(14, 4*rows), sharey=True)
    if rows == 1:
        axes = [axes]  # normalize to 2D indexing
    for r, (label, ex) in enumerate(cases):
        p, (t1, proj1), (t2, proj2), (a1, a2) = ex
        for c, (projs, qtitle) in enumerate([(proj1, 'Q1'), (proj2, 'Q2')]):
            ax = axes[r][c]
            ax.plot(range(len(projs)), projs, marker='o', ms=3, lw=1)
            ax.axhline(0, color='k', lw=0.8, alpha=0.6)
            ax.set_title(qtitle)
            ax.set_xlabel('Generated token index')
            ax.set_ylabel('Projection onto v_faithful')
        if hasattr(p, 'left'):
            pair_str = '{} | {}'.format(getattr(p, 'left', '?'), getattr(p, 'right', '?'))
        else:
            pair_str = '{},{}'.format(getattr(p, 'x', '?'), getattr(p, 'y', '?'))
        axes[r][0].text(0.02, 0.95, '{}: {} | answers: {}, {}'.format(label, pair_str, a1, a2), transform=axes[r][0].transAxes, fontsize=10, va='top', ha='left')
    plt.tight_layout()
    plt.show()

plot_grid(faithful_ex, unfaithful_ex)

NameError: name 'faithful_ex' is not defined